In [ ]:
# xgboost
#https://xgboost.readthedocs.io/en/latest/python/index.html
# https://github.com/dmlc/xgboost/blob/master/demo/multiclass_classification/train.py#L35

In [2]:
!ls 

load_mnist.py              train-images-idx3-ubyte.gz
t10k-images-idx3-ubyte.gz  train-labels-idx1-ubyte.gz
t10k-labels-idx1-ubyte.gz  xgboost.ipynb
tensorflow.ipynb


In [56]:
from load_mnist import load_mnist
import xgboost as xgb
import numpy as np

In [40]:
train_data, train_label = load_mnist("","train")
test_data,test_label = load_mnist("","t10k")

In [47]:
dtrain = xgb.DMatrix(train_data, label=train_label)
dtest = xgb.DMatrix(test_data, label=test_label)


In [60]:
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 10
param['silent'] = 1
param['nthread'] = 6
param['num_class'] = 10

In [61]:
param

{'eta': 0.1,
 'max_depth': 10,
 'nthread': 6,
 'num_class': 10,
 'objective': 'multi:softmax',
 'silent': 1}

In [62]:
evallist = [(dtrain, 'train'),(dtest, 'test')]

In [63]:
num_round = 10
bst = xgb.train(param, dtrain, num_round, evallist)

[0]	train-merror:0.086567	test-merror:0.172
[1]	train-merror:0.06925	test-merror:0.1512
[2]	train-merror:0.06165	test-merror:0.1439
[3]	train-merror:0.0578	test-merror:0.1402
[4]	train-merror:0.05435	test-merror:0.1399
[5]	train-merror:0.051517	test-merror:0.1402
[6]	train-merror:0.048417	test-merror:0.1388
[7]	train-merror:0.04605	test-merror:0.1373
[8]	train-merror:0.044333	test-merror:0.136
[9]	train-merror:0.042167	test-merror:0.1364


In [66]:
pred = bst.predict(dtest)

In [68]:
acc_rate = np.sum(pred == test_label) / test_label.shape[0]
print('Test accuracy using softmax = {}'.format(acc_rate))

Test accuracy using softmax = 0.8636
